In [1]:
from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os 


load_dotenv()
os.getenv("GOOGLE_API_KEY") 
my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [2]:
from langchain_google_genai.chat_models import  ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model= "gemini-1.5-flash")

In [4]:
from langchain_core.messages import HumanMessage
import base64
import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

In [5]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = llm.invoke([message])
print(response.content)

The weather looks sunny and clear with a bright blue sky and fluffy white clouds. It appears to be a warm day with no rain.  The tall grass suggests that it is likely summer.


We can feed the image URL directly in a content block of type "image_url". Note that only some model providers support this.

In [7]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = llm.invoke([message])
print(response.content)

The weather looks sunny and clear. The sky is a bright blue with fluffy white clouds. The grass is green and lush, suggesting that it is a warm day. There is no indication of rain or wind in the image.


We can also pass in multiple images.

In [9]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "are these two images the same?"},
        {"type": "image_url", "image_url": {"url": image_url}},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = llm.invoke([message])
print(response.content)

Yes, the two images are the same.


In [11]:
from typing import Literal

from langchain_core.tools import tool

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

@tool
def weather_tool(weather: Literal["sunny", "cloudy", "rainy"]) -> None:
    """Describe the weather"""
    pass


model_with_tools = llm.bind_tools([weather_tool])

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model_with_tools.invoke([message])
print(response.tool_calls)

[]


##### Using multimodal prompts

In [12]:
import base64

import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

In [14]:
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Describe the image provided"),
        (
            "user",
            [
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                }
            ],
        ),
    ]
)

In [16]:
chain = prompt | llm

In [17]:
response = chain.invoke({"image_data": image_data})
print(response.content)

The image shows a wooden boardwalk stretching through a field of tall green grass. The boardwalk is centered in the image and leads to a distant line of trees. The sky is a bright blue with wispy white clouds. The grass is tall and lush, and the overall feeling of the image is one of peace and tranquility.


In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "compare the two pictures provided"),
        (
            "user",
            [
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data1}"},
                },
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data2}"},
                },
            ],
        ),
    ]
)

In [19]:
chain = prompt | llm
response = chain.invoke({"image_data1": image_data, "image_data2": image_data})
print(response.content)

The two pictures are identical.
